# SQLite 使用入门教程

SQLite是一个轻量级的、自给自足的、零配置的、事务性的SQL数据库引擎。它是世界上使用最广泛的数据库引擎，被内嵌到无数的应用程序中，包括手机、浏览器、操作系统等。

本教程将介绍如何使用Python的`sqlite3`模块来操作SQLite数据库。

## 1. 导入所需库

Python标准库中自带了`sqlite3`模块，无需额外安装。

In [1]:
import sqlite3
import pandas as pd
from datetime import datetime

print("SQLite版本:", sqlite3.version)
print("SQLite库版本:", sqlite3.sqlite_version)

SQLite版本: 2.6.0
SQLite库版本: 3.45.3


## 2. 创建/连接到数据库

使用`sqlite3.connect()`方法来创建一个新的数据库或连接到一个已存在的数据库。

In [5]:
# 创建一个内存数据库（数据会在连接关闭后丢失）
conn = sqlite3.connect(':memory:')

# 创建或连接到一个文件数据库
# conn = sqlite3.connect('example.db')
print("已成功连接到数据库")


已成功连接到数据库


## 3. 创建表

使用`CREATE TABLE`语句来创建一个新表。我们将创建一个用户表作为示例。

In [6]:
# 获取游标对象
cursor = conn.cursor()

# 创建users表
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT, -- 用户ID，主键，自动增长
    name TEXT NOT NULL,                   -- 用户名，不能为空
    age INTEGER,                          -- 年龄
    email TEXT UNIQUE,                    -- 邮箱，唯一约束
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP -- 创建时间，默认为当前时间戳
)
''')

# 提交事务
conn.commit()
print("用户表创建成功")

用户表创建成功


## 4. 插入数据

使用`INSERT INTO`语句向表中插入数据。

In [7]:
# 插入单条数据
cursor.execute('''
INSERT INTO users (name, age, email) VALUES (?, ?, ?)
''', ('张三', 25, 'zhangsan@example.com'))

# 插入多条数据
users_data = [
    ('李四', 30, 'lisi@example.com'),
    ('王五', 22, 'wangwu@example.com'),
    ('赵六', 35, 'zhaoliu@example.com')
]

cursor.executemany('''
INSERT INTO users (name, age, email) VALUES (?, ?, ?)
''', users_data)

conn.commit()
print(f"成功插入了{cursor.rowcount}条数据")

成功插入了3条数据


## 5. 查询数据

使用`SELECT`语句从表中查询数据。

In [8]:
# 查询所有用户
cursor.execute('SELECT * FROM users')
all_users = cursor.fetchall()

print("所有用户:")
for user in all_users:
    print(user)

所有用户:
(1, '张三', 25, 'zhangsan@example.com', '2025-05-08 02:41:40')
(2, '李四', 30, 'lisi@example.com', '2025-05-08 02:41:40')
(3, '王五', 22, 'wangwu@example.com', '2025-05-08 02:41:40')
(4, '赵六', 35, 'zhaoliu@example.com', '2025-05-08 02:41:40')


In [9]:
# 使用pandas展示查询结果
cursor.execute('SELECT * FROM users')
column_names = [description[0] for description in cursor.description]
df = pd.DataFrame(cursor.fetchall(), columns=column_names)
df

,id,name,age,email,created_at
0,1,张三,25,zhangsan@example.com,2025-05-08 02:41:40
1,2,李四,30,lisi@example.com,2025-05-08 02:41:40
2,3,王五,22,wangwu@example.com,2025-05-08 02:41:40
3,4,赵六,35,zhaoliu@example.com,2025-05-08 02:41:40


In [10]:
# 条件查询
cursor.execute('SELECT name, email FROM users WHERE age > ?', (25,))
filtered_users = cursor.fetchall()

print("年龄大于25岁的用户:")
for user in filtered_users:
    print(user)

年龄大于25岁的用户:
('李四', 'lisi@example.com')
('赵六', 'zhaoliu@example.com')


## 6. 更新数据

使用`UPDATE`语句修改表中的数据。

In [11]:
# 更新用户年龄
cursor.execute('''
UPDATE users SET age = ? WHERE name = ?
''', (26, '张三'))

conn.commit()
print(f"更新了{cursor.rowcount}条记录")

# 查看更新后的数据
cursor.execute('SELECT * FROM users WHERE name = ?', ('张三',))
updated_user = cursor.fetchone()
print("更新后的用户信息:", updated_user)

更新了1条记录
更新后的用户信息: (1, '张三', 26, 'zhangsan@example.com', '2025-05-08 02:41:40')


## 7. 删除数据

使用`DELETE`语句从表中删除数据。

In [12]:
# 删除特定用户
cursor.execute('DELETE FROM users WHERE name = ?', ('赵六',))
conn.commit()
print(f"删除了{cursor.rowcount}条记录")

# 查看剩余用户
cursor.execute('SELECT * FROM users')
remaining_users = cursor.fetchall()
print("剩余用户数量:", len(remaining_users))

# 使用pandas展示查询结果
column_names = [description[0] for description in cursor.description]
pd.DataFrame(remaining_users, columns=column_names)

删除了1条记录
剩余用户数量: 3


,id,name,age,email,created_at
0,1,张三,26,zhangsan@example.com,2025-05-08 02:41:40
1,2,李四,30,lisi@example.com,2025-05-08 02:41:40
2,3,王五,22,wangwu@example.com,2025-05-08 02:41:40


## 8. 事务处理

SQLite默认启用事务。每个修改数据库的SQL语句开始一个事务，并且事务持续到被显式提交或回滚。

In [13]:
try:
    # 开始一个事务（SQLite中默认就是这样）
    cursor.execute("INSERT INTO users (name, age, email) VALUES (?, ?, ?)", 
                   ('田七', 40, 'tianqi@example.com'))
    cursor.execute("INSERT INTO users (name, age, email) VALUES (?, ?, ?)", 
                   ('孙八', 45, 'sunba@example.com'))
    
    # 故意制造一个错误：插入重复的email
    # cursor.execute("INSERT INTO users (name, age, email) VALUES (?, ?, ?)", 
    #                ('错误用户', 20, 'tianqi@example.com'))
    
    # 提交事务
    conn.commit()
    print("事务提交成功")
except sqlite3.Error as e:
    # 如果发生错误，回滚所有更改
    conn.rollback()
    print(f"事务回滚，错误: {e}")

事务提交成功


## 9. 创建索引

使用索引可以加快查询速度。

In [14]:
# 为name字段创建索引
cursor.execute('''
CREATE INDEX IF NOT EXISTS idx_users_name ON users (name)
''')

conn.commit()
print("索引创建成功")

索引创建成功


## 10. 多表操作与连接查询

创建多个相关联的表，并使用连接查询。

In [15]:
# 创建订单表
cursor.execute('''
CREATE TABLE IF NOT EXISTS orders (
    id INTEGER PRIMARY KEY AUTOINCREMENT, -- 订单ID，主键，自动增长
    user_id INTEGER NOT NULL,             -- 用户ID，外键，关联users表，不能为空
    product_name TEXT NOT NULL,           -- 商品名称，不能为空
    price REAL NOT NULL,                  -- 商品价格，不能为空
    order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP, -- 订单日期，默认为当前时间戳
    FOREIGN KEY (user_id) REFERENCES users (id) -- 设置外键约束，关联users表的id字段
)
''')

# 准备要插入的订单数据
# 每个元组代表一条订单记录：(用户ID, 商品名称, 价格)
order_data = [
    (1, '手机', 3999.00),  # 假设用户ID为1的用户（张三）购买了手机
    (1, '耳机', 299.00),   # 假设用户ID为1的用户（张三）购买了耳机
    (2, '笔记本', 6999.00), # 假设用户ID为2的用户（李四）购买了笔记本
    (3, '平板', 2999.00)   # 假设用户ID为3的用户（王五）购买了平板
]

# 批量插入订单数据
# 使用参数化查询防止SQL注入，并高效插入多条记录
cursor.executemany('''
INSERT INTO orders (user_id, product_name, price) VALUES (?, ?, ?)
''', order_data)

# 提交事务，将更改保存到数据库
conn.commit()
print("订单表创建并插入数据成功")

订单表创建并插入数据成功


In [16]:
# 使用JOIN查询用户及其订单
cursor.execute('''
SELECT 
    u.name,             -- 用户名
    u.email,            -- 用户邮箱
    o.product_name,     -- 商品名称
    o.price,            -- 商品价格
    o.order_date        -- 订单日期
FROM 
    users u             -- 从users表（别名u）
JOIN 
    orders o            -- 连接orders表（别名o）
ON 
    u.id = o.user_id    -- 连接条件：users表的id等于orders表的user_id
ORDER BY 
    u.name,             -- 按用户名排序
    o.order_date        -- 然后按订单日期排序
''')

join_results = cursor.fetchall()
column_names = ['用户名', '邮箱', '商品名', '价格', '订单日期']
pd.DataFrame(join_results, columns=column_names)

,用户名,邮箱,商品名,价格,订单日期
0,张三,zhangsan@example.com,手机,3999.0,2025-05-08 02:43:42
1,张三,zhangsan@example.com,耳机,299.0,2025-05-08 02:43:42
2,李四,lisi@example.com,笔记本,6999.0,2025-05-08 02:43:42
3,王五,wangwu@example.com,平板,2999.0,2025-05-08 02:43:42


In [17]:
# 聚合查询：计算每个用户的订单总额
cursor.execute('''
SELECT 
    u.name,                               -- 用户名
    COUNT(o.id) as order_count,           -- 计算每个用户的订单数量，别名为order_count
    SUM(o.price) as total_spent           -- 计算每个用户的总消费金额，别名为total_spent
FROM 
    users u                               -- 从users表（别名u）
LEFT JOIN 
    orders o ON u.id = o.user_id          -- 左连接orders表（别名o），连接条件是users表的id等于orders表的user_id
                                          -- 使用LEFT JOIN确保即使没有订单的用户也会被包含在结果中
GROUP BY 
    u.id                                  -- 按用户ID分组，这样聚合函数（COUNT, SUM）会对每个用户分别计算
ORDER BY 
    total_spent DESC                      -- 按总消费金额降序排序
''')

summary_results = cursor.fetchall()
column_names = ['用户名', '订单数量', '消费总额']
pd.DataFrame(summary_results, columns=column_names)

,用户名,订单数量,消费总额
0,李四,1,6999.0
1,张三,2,4298.0
2,王五,1,2999.0
3,田七,0,NaN
4,孙八,0,NaN


## 11. 使用上下文管理器与连接池

在实际应用中，推荐使用上下文管理器来自动关闭连接。

In [21]:
# 使用上下文管理器来确保数据库连接在使用完毕后自动关闭
# 这会连接到 'example.db' 文件数据库。如果文件不存在，则会创建它。
# 这是一个基于文件的数据库，与之前单元格中使用的内存数据库 'conn' 是分开的。
with sqlite3.connect('example.db') as conn: # 此处的 'conn' 是 'with' 语句块的局部变量，指向 'example.db'
    cursor = conn.cursor() # 此处的 'cursor' 也是局部的，用于 'example.db' 的连接

    # 在 'example.db' 中创建 'users' 表，如果它尚不存在。
    # 表结构与内存数据库中使用的结构相匹配。
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS users ( -- 如果 'users' 表不存在，则创建该表
        id INTEGER PRIMARY KEY AUTOINCREMENT, -- 用户ID，整数类型，主键，自动增长
        name TEXT NOT NULL,                   -- 用户名，文本类型，不能为空
        age INTEGER,                          -- 年龄，整数类型
        email TEXT UNIQUE,                    -- 邮箱，文本类型，必须唯一
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP -- 创建时间，时间戳类型，默认为当前时间戳
    )
    ''')

    # 为文件数据库准备的用户数据列表
    # 向 'example.db' 的 'users' 表中插入一些示例数据。
    # 使用 'INSERT OR IGNORE' 确保如果多次运行此单元格，
    # 不会尝试插入违反邮箱唯一约束的重复条目。
    users_data_for_file_db = [
        ('File User Alice', 30, 'alice.file@example.com'),
        ('File User Bob', 24, 'bob.file@example.com')
    ]
    cursor.executemany('''
    INSERT OR IGNORE INTO users (name, age, email) VALUES (?, ?, ?) -- 批量插入数据到 'users' 表。如果数据导致唯一约束冲突（如email重复），则忽略该条插入。
    ''', users_data_for_file_db)

    conn.commit() # 提交事务，将表创建和数据插入操作永久保存到 'example.db'

    # 现在，执行查询以计算 'example.db' 中 'users' 表的记录总数
    cursor.execute('SELECT COUNT(*) FROM users') # 从 'users' 表中统计所有行数 (用户总数)
    count = cursor.fetchone()[0] # 获取查询结果的第一行第一列（即用户总数）
    print(f"总用户数 (in example.db): {count}") # 打印 'example.db' 中的用户总数

# 当退出 'with' 语句块时，到 'example.db' 的连接 'conn' 会自动关闭。

总用户数 (in example.db): 2


## 12. 导出数据库结构和数据

可以导出数据库的结构和数据，便于备份或迁移。

In [22]:
def dump_db(conn):
    """
    导出数据库的结构（CREATE TABLE语句）和数据（INSERT语句和数据元组）。

    参数:
    conn (sqlite3.Connection): 一个已连接的SQLite数据库连接对象。

    返回:
    list: 一个列表，其中包含数据库的CREATE TABLE语句（字符串）
          以及 (INSERT INTO语句模板, 数据元组) 形式的元组。
    """
    # 获取数据库连接的游标对象，用于执行SQL语句
    cursor = conn.cursor()
    
    # 查询sqlite_master表以获取数据库中所有用户定义的表的名称
    # sqlite_master是SQLite内部表，存储数据库的元信息
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()  # 获取所有查询结果，每个结果是一个包含表名的元组
    
    dump = []  # 初始化一个空列表，用于存储导出的SQL语句和数据
    
    # 遍历查询到的每个表
    for table_name_tuple in tables:
        table = table_name_tuple[0]  # 从元组中提取表名字符串
        
        # 跳过SQLite内部表（通常以'sqlite_'开头），我们只关心用户创建的表
        if table.startswith('sqlite_'):
            continue
            
        # 获取表的结构信息（列名、类型、约束等）
        # PRAGMA table_info(table_name) 返回指定表的每一列的信息
        cursor.execute(f"PRAGMA table_info({table})")
        columns = cursor.fetchall()  # 获取所有列的信息
        
        # 开始构建CREATE TABLE语句
        create_statement = f"CREATE TABLE {table} (\n"
        column_definitions = [] # 用于存储每一列的定义字符串
        for col in columns:
            # col[0]: cid (列ID)
            # col[1]: name (列名)
            # col[2]: type (数据类型)
            # col[3]: notnull (是否非空，1表示非空，0表示可空)
            # col[4]: dflt_value (默认值)
            # col[5]: pk (是否主键，1表示是主键的一部分，0表示不是)
            col_def = f"    {col[1]} {col[2]}" # 列名和数据类型
            if col[3]:  # 如果notnull为1 (True)
                col_def += " NOT NULL" # 添加非空约束
            if col[5]:  # 如果pk为1 (True)
                col_def += " PRIMARY KEY" # 添加主键约束
            if col[4] is not None: # 如果存在默认值
                # 对于文本类型的默认值，需要用单引号括起来
                if isinstance(col[4], str):
                    col_def += f" DEFAULT '{col[4]}'"
                else:
                    col_def += f" DEFAULT {col[4]}"
            column_definitions.append(col_def)
        
        create_statement += ",\n".join(column_definitions) # 用逗号和换行符连接所有列的定义
        create_statement += "\n);" # 结束CREATE TABLE语句
        dump.append(create_statement) # 将构建好的CREATE TABLE语句添加到导出列表中
        
        # 获取表中的所有数据
        cursor.execute(f"SELECT * FROM {table}")
        rows = cursor.fetchall()  # 获取表中的所有行数据
        
        # 为表中的每一行数据构建INSERT语句模板和数据元组
        if rows: # 只有当表中有数据时才生成INSERT语句
            # 创建一个通用的INSERT语句模板，使用占位符 '?'
            # 占位符的数量与列的数量相同
            placeholders = ', '.join(['?' for _ in rows[0]]) # 根据第一行数据的列数生成占位符
            insert_template = f"INSERT INTO {table} VALUES ({placeholders});"
            for row in rows:
                # 将INSERT语句模板和对应的行数据元组添加到导出列表中
                # 这样做是为了方便后续使用 executemany 或带参数的 execute 执行
                dump.append((insert_template, row)) 
    
    return dump # 返回包含所有导出信息的列表

# 使用之前在 notebook 中定义的 'conn' 连接对象（连接到内存数据库）来导出数据库结构和数据
# 'conn' 是在 CELL 4 中定义的 :memory: 数据库连接
dump_result = dump_db(conn)

print("数据库结构和数据导出示例 (内存数据库中的 'users' 和 'orders' 表):")
# 遍历导出结果的前5条记录并打印
# 这可能包括CREATE TABLE语句或 (INSERT语句, 数据) 元组
for item in dump_result[:5]:  
    print(item)

数据库结构和数据导出示例 (内存数据库中的 'users' 和 'orders' 表):
CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER,
    email TEXT,
    created_at TIMESTAMP DEFAULT 'CURRENT_TIMESTAMP'
);
('INSERT INTO users VALUES (?, ?, ?, ?, ?);', (1, 'File User Alice', 30, 'alice.file@example.com', '2025-05-08 02:53:46'))
('INSERT INTO users VALUES (?, ?, ?, ?, ?);', (2, 'File User Bob', 24, 'bob.file@example.com', '2025-05-08 02:53:46'))


## 13. 关闭连接

使用完数据库后，关闭连接释放资源。

In [23]:
# 关闭游标
cursor.close()

# 关闭连接
conn.close()
print("数据库连接已关闭")

数据库连接已关闭


## 总结

本教程介绍了SQLite的基本操作，包括：

1. 创建/连接数据库
2. 创建表
3. 插入数据
4. 查询数据
5. 更新数据
6. 删除数据
7. 事务处理
8. 创建索引
9. 多表连接查询
10. 使用上下文管理器
11. 导出数据库结构和数据

SQLite因其轻量级、无需服务器配置且性能良好的特点，非常适合用于小型应用、开发和测试环境。了解并掌握这些基础操作可以帮助你更高效地使用SQLite数据库。